<a href="https://colab.research.google.com/github/chloebh9/Recommendation-system/blob/main/L03_1_Rating_Prediction_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-25m.zip
!unzip ml-25m.zip

--2024-04-10 06:02:28--  https://files.grouplens.org/datasets/movielens/ml-25m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261978986 (250M) [application/zip]
Saving to: ‘ml-25m.zip’

ml-25m.zip          100%[===================>] 249.84M  35.9MB/s    in 6.5s    

2024-04-10 06:02:35 (38.4 MB/s) - ‘ml-25m.zip’ saved [261978986/261978986]

Archive:  ml-25m.zip
   creating: ml-25m/
  inflating: ml-25m/tags.csv         
  inflating: ml-25m/links.csv        
  inflating: ml-25m/README.txt       
  inflating: ml-25m/ratings.csv      
  inflating: ml-25m/genome-tags.csv  
  inflating: ml-25m/genome-scores.csv  
  inflating: ml-25m/movies.csv       


In [ ]:
import numpy as np

with open("ml-25m/ratings.csv", "r") as f:
    print(f.readline()) # skip column names

    users = []
    items = []
    ratings = []

    for line in f:
        uid, mid, rating, timestamp = line.split(",")
        users.append(int(uid))
        items.append(int(mid))
        ratings.append(float(rating))

    users = np.array(users)
    items = np.array(items)
    ratings = np.array(ratings)

userId,movieId,rating,timestamp



In [ ]:
rmse = ((ratings - ratings.mean()) ** 2).mean() ** 0.5
print(rmse)

1.0607439399275531


In [ ]:
alpha = ratings.mean()
user_bias = np.zeros(users.max() + 1)
item_bias = np.zeros(items.max() + 1)

In [ ]:
lr = 1
lmd = 0.001

n_ratings = len(ratings)
n_users = len(user_bias)
n_items = len(item_bias)

for epoch in range(20):

    h = alpha + user_bias[users] + item_bias[items]
    diff = h - ratings

    rmse = (diff ** 2).mean() ** 0.5
    print(f"epoch: {epoch}, rmse: {rmse}")

    grd_alpha = diff.mean()
    grd_user_bias = np.bincount(users, weights=diff)/n_ratings + lmd * user_bias/n_users
    grd_item_bias = np.bincount(items, weights=diff)/n_ratings + lmd * item_bias/n_items

    alpha = alpha - lr * grd_alpha
    user_bias = user_bias - lr * grd_user_bias
    item_bias = item_bias - lr * grd_item_bias

h = alpha + user_bias[users] + item_bias[items]
diff = h - ratings

rmse = (diff ** 2).mean() ** 0.5
print(f"rmse: {rmse}, alpha: {alpha}")

epoch: 0, rmse: 1.0607439399275531
epoch: 1, rmse: 1.0606227041512328
epoch: 2, rmse: 1.0605018083005335
epoch: 3, rmse: 1.0603812424803158
epoch: 4, rmse: 1.0602610053766328
epoch: 5, rmse: 1.060141095681595
epoch: 6, rmse: 1.0600215120933394
epoch: 7, rmse: 1.0599022533159907
epoch: 8, rmse: 1.0597833180596394
epoch: 9, rmse: 1.0596647050403154
epoch: 10, rmse: 1.0595464129799474
epoch: 11, rmse: 1.0594284406063486
epoch: 12, rmse: 1.059310786653174
epoch: 13, rmse: 1.0591934498599036
epoch: 14, rmse: 1.0590764289718062
epoch: 15, rmse: 1.0589597227399157
epoch: 16, rmse: 1.0588433299209998
epoch: 17, rmse: 1.0587272492775308
epoch: 18, rmse: 1.0586114795776682
epoch: 19, rmse: 1.0584960195952156
rmse: 1.0583808681095987, alpha: 3.5313318465676904


In [ ]:
import torch
ratings_tensor = torch.from_numpy(ratings)

alpha = torch.tensor(ratings.mean())
alpha.requires_grad_(True)
user_bias = torch.zeros(users.max() + 1, requires_grad=True)
item_bias = torch.zeros(items.max() + 1, requires_grad=True)

optim = torch.optim.Adam([alpha, user_bias, item_bias], lr=0.3)

lmd = 0.001

for epoch in range(20):
    h = alpha + user_bias[users] + item_bias[items]
    mse = ((h - ratings_tensor) ** 2).mean()
    reg = lmd * ((item_bias ** 2).mean() + (user_bias ** 2).mean())
    cost = mse + reg

    optim.zero_grad()
    cost.backward()
    optim.step()

    # with torch.no_grad():
    #     print((mse ** 0.5).item(), alpha.item())

    with torch.no_grad():
        rmse = ((h - ratings_tensor) ** 2).mean() ** 0.5
        print(f"epoch: {epoch}, rmse: {rmse}")

epoch: 0, rmse: 1.0607439399275533
epoch: 1, rmse: 0.9669325508290363
epoch: 2, rmse: 0.9497903759085393
epoch: 3, rmse: 0.9171939835020677
epoch: 4, rmse: 0.8941977578397844
epoch: 5, rmse: 0.9097900925389514
epoch: 6, rmse: 0.9059687935173155
epoch: 7, rmse: 0.8945284306008058
epoch: 8, rmse: 0.8960262980198718
epoch: 9, rmse: 0.892189653674854
epoch: 10, rmse: 0.8781706889731724
epoch: 11, rmse: 0.8717876177492263
epoch: 12, rmse: 0.8759573490807723
epoch: 13, rmse: 0.8761336781525304
epoch: 14, rmse: 0.8709138637513829
epoch: 15, rmse: 0.8687833630096475
epoch: 16, rmse: 0.8684808875097653
epoch: 17, rmse: 0.8644695056645418
epoch: 18, rmse: 0.8603502917327274
epoch: 19, rmse: 0.8612059545815784


In [ ]:
from tqdm import tqdm
lmd = 0.001

alpha = ratings.mean()
user_bias = np.zeros(users.max() + 1)
item_bias = np.zeros(items.max() + 1)

for epoch in range(10):
    h = alpha + user_bias[users] + item_bias[items]
    rmse = ((h - ratings) ** 2).mean() ** 0.5
    print(rmse)

    alpha = (ratings - (user_bias[users] + item_bias[items])).mean()
    user_bias = np.bincount(users, weights=ratings - (alpha + item_bias[items])) / (np.bincount(users) + lmd)
    item_bias = np.bincount(items, weights=ratings - (alpha + user_bias[users])) / (np.bincount(items) + lmd)

h = alpha + user_bias[users] + item_bias[items]
rmse = ((h - ratings) ** 2).mean() ** 0.5
print(f"final rmse: {rmse}")

1.0607439399275531
0.8663159834756426
0.8510867185839471
0.8503568292676987
0.8503078899843288
0.8503025377563744
0.8503016450633218
0.8503014646716495
0.8503014258804191
0.8503014173907169
final rmse: 0.8503014155289849
